In [1]:
import mlearn_cuda as mlearn
from mlearn_cuda import Module
from mlearn_cuda import layers
from mlearn_cuda.optimizer import RMSProp,SGD
from mlearn_cuda.utils import DataLoader,pre_F
from mlearn_cuda import functional as F
from torchvision import datasets
import cupy as cp
import sys

In [2]:
train = datasets.MNIST("datasets/",train=True,download=True)
test = datasets.MNIST("datasets/",train=False,download=True)

In [3]:
pre = [pre_F.normalize_MinMax]
trainset = DataLoader((train.data,train.targets),batch_size=32, shuffle=True, preprocessing = pre)
testset = DataLoader((test.data,test.targets),batch_size=32,shuffle=True, preprocessing=pre)

CUDARuntimeError: cudaErrorUnknown: unknown error

In [ ]:
class Net(mlearn.Module):
    def __init__(self):
        self.dense1 = layers.Dense(784,300)
        self.dense2 = layers.Dense(300,150)
        self.dense3 = layers.Dense(150,10)
        
    def forward(self, inputs):
        o = self.dense1(inputs)
        o = F.relu(o)
        o = self.dense2(o)
        o = F.tanh(o)
        o = self.dense3(o)
        return (F.tanh(o))
net = Net()

In [ ]:
hist = []
import numpy as np
optimizer = SGD(net,0.0005)
EPOCHS = 5
net.zero_grad()
for epoch in range(EPOCHS):
    running_loss = 0.0
    bar = " "*20
    for i, batch in enumerate(trainset, 0):
        features, labels = batch
        net.zero_grad()
        predict = net(features.reshape(-1,784))
        loss = F.cross_entropy(predict,labels)
        loss.backward()
        shapes = optimizer.step()
        running_loss += loss.data
        print(f"\r{epoch+1}/{EPOCHS} Batch %-4d/1874  [{bar}] -> Loss %.5f"%(i,loss.data), end="")
        sys.stdout.flush()
    print()
print('trainning completed!')

In [ ]:
correct = 0
total = 0
features = 0
labels = 0
for batch in testset:
    features, labels = batch
    o = net(features.reshape(-1,784))
    predict = []
    for x in o.data:
        predict.append(cp.argmax(x))
    for b in predict == cp.asnumpy(labels.data):
        if b:
            correct += 1
    total += 32
print("Accuracy %.5f"%(correct / total))